In [27]:
%pip install pandas

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [49]:
%pip install xgboost
%pip install scikit-learn

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import xgboost as xgb

# 1. Data 다운로드 & 로드

In [50]:
users = pd.read_csv('../data/Users.csv')
books = pd.read_csv('../data/Books.csv')
ratings = pd.read_csv('../data/Ratings.csv')

# users = pd.read_csv('../data/Users.csv', dtype={'Year-Of-Publication': 'object'}, low_memory=False)
# books = pd.read_csv('../data/Books.csv', dtype={'Year-Of-Publication': 'object'}, low_memory=False)
# ratings = pd.read_csv('../data/Ratings.csv', dtype={'Year-Of-Publication': 'object'}, low_memory=False)

/var/folders/92/pn1dsn_x7db99lb6q2_7l2c40000gn/T/ipykernel_28919/2457756711.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('../data/Books.csv')


In [30]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [31]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [32]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


# 2. 전처리: 유저 데이터

In [33]:
# Let's slice our data to 5k users for demo purposes.
users = users.iloc[5000:10_000]
users.head()

,User-ID,Location,Age
5000,5001,"inverness, california, usa",NaN
5001,5002,"milano, milano, italy",NaN
5002,5003,"snohomish, washington, usa",NaN
5003,5004,"valència, país valencià, spain",23.0
5004,5005,"olesa de montserrat, barcelona, spain",17.0


In [34]:
# 결측치 확인 및 제거 
# isna()로 확인 후, 유저의 평균 나이로 Age 빈 자리를 채워준다. (결측치 처리)
users.isna().sum()
users["Age"].fillna(users["Age"].mean(), inplace=True)
# location: 상세 주소를 ',' 단위로 나눠서 가장 마지막 값(국가)만 남긴다.
users["Location"] = users["Location"].apply(lambda x: x.split(", ")[-1])
users.head()

,User-ID,Location,Age
5000,5001,usa,32.034934
5001,5002,italy,32.034934
5002,5003,usa,32.034934
5003,5004,spain,23.000000
5004,5005,spain,17.000000


In [35]:
# 5개 국가의 유저들로만 숫자를 줄인다 (데모를 위한 성능 향상)  

# 국가별 user 집계. user 데이터프레임의 location 칼럼을 순서대로 집계합니다.  
loc_counts = users.Location.value_counts()
loc_counts

# 사용자가 256명보다 작은 국가는 location에 other로 표기한 후, 
users["Location"] = users["Location"].apply(lambda x: "other" if loc_counts[x] < 256 else x)

#location이 other인 데이터를 데이터프레임에서 제외합니다
users = users[users["Location"] != "other"]
users.head()

,User-ID,Location,Age
5000,5001,usa,32.034934
5001,5002,italy,32.034934
5002,5003,usa,32.034934
5003,5004,spain,23.000000
5004,5005,spain,17.000000


In [36]:
# one-hot encoding 적용 
users = pd.get_dummies(users, columns=["Location"])

# 데이터 확인
users.head()
users.shape

#Save rating only for present users
ratings = ratings[ratings["User-ID"].isin(users["User-ID"])]
ratings.shape

(14758, 3)

# 3. 전처리: 책 데이터

In [37]:
# Let's move to the books dataset. For demo purposes we will take only 5k books and 1 feature Year-Of-Publication.

train_df = pd.merge(ratings, users, on="User-ID")
train_df = pd.merge(books, train_df, on="ISBN")
train_df = train_df[["Year-Of-Publication", "Book-Rating", "Age", "Location_canada", "Location_italy", "Location_spain", "Location_united kingdom", "Location_usa"]]
train_df.columns = ["book_publication_year", "rating", "user_age", "user_in_canada", "user_in_italy", "user_in_spain", "user_in_united_kingdom", "user_in_usa"]
train_df["book_publication_year"] = train_df["book_publication_year"].astype(int)

In [38]:
train_df.head()

,book_publication_year,rating,user_age,user_in_canada,user_in_italy,user_in_spain,user_in_united_kingdom,user_in_usa
0,1999,0,31.000000,False,False,False,False,True
1,1999,9,49.000000,False,False,False,False,True
2,1999,8,53.000000,False,False,False,False,True
3,1999,5,32.034934,False,False,False,False,True
4,1999,0,32.034934,False,False,False,False,True


In [39]:
train_df.shape

(13589, 8)

In [40]:
train_df["liked"] = (train_df["rating"] >= 7).astype(int)
train_df = train_df.drop("rating", axis=1)
train_df.head()

,book_publication_year,user_age,user_in_canada,user_in_italy,user_in_spain,user_in_united_kingdom,user_in_usa,liked
0,1999,31.000000,False,False,False,False,True,0
1,1999,49.000000,False,False,False,False,True,1
2,1999,53.000000,False,False,False,False,True,1
3,1999,32.034934,False,False,False,False,True,0
4,1999,32.034934,False,False,False,False,True,0


In [41]:
train_df.liked.value_counts()

liked
0    9264
1    4325
Name: count, dtype: int64

In [42]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_df[["book_publication_year", "user_age"]] = scaler.fit_transform(train_df[["book_publication_year", "user_age"]])

# 4. Train, Test 데이터셋 만들기 

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_df.drop("liked", axis=1), 
    train_df["liked"], 
    stratify=train_df["liked"],
    test_size=0.2,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=42)

# 5. 모델 학습시키기

In [44]:
model = xgb.XGBRegressor(
    n_estimators=10,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-rmse:0.46478
[1]	validation_0-rmse:0.46389
[2]	validation_0-rmse:0.46280
[3]	validation_0-rmse:0.46192
[4]	validation_0-rmse:0.46117
[5]	validation_0-rmse:0.46082
[6]	validation_0-rmse:0.46034
[7]	validation_0-rmse:0.46028
[8]	validation_0-rmse:0.46000
[9]	validation_0-rmse:0.45999


/Users/ijong-won/.pyenv/versions/3.8.3/lib/python3.8/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# 6. 모델 평가하기

In [45]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
y_pred = (y_pred >= 0.5).astype(int)
accuracy_score(y_test, y_pred)

0.681383370125092

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      1.00      0.81      1853
           1       0.46      0.01      0.01       865

    accuracy                           0.68      2718
   macro avg       0.57      0.50      0.41      2718
weighted avg       0.61      0.68      0.56      2718



# 7. 모델 export

In [47]:
import pickle
pickle.dump(model, open("../artifacts/model.pkl", "wb"))

# 8. Save test data 

In [48]:
test = X_test.sample(4)
test["liked"] = y_test[test.index]
test.to_csv("../artifacts/test.csv", index=False)